In [1]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import ChatTTS
from IPython.display import Audio
chat = ChatTTS.Chat()
chat.load_models()
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

/opt/anaconda3/envs/toxic_classification/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:ChatTTS.core:Load from cache: /Users/tal/.cache/huggingface/hub/models--2Noise--ChatTTS/snapshots/cc14302f34d7855eb3420d1fd48345012ff1460d
INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.


音色分支-A:自定义音色

In [2]:
# 保存音频文件
import numpy as np
from scipy.io.wavfile import write

# 假设wavs[0]是包含音频数据的二维数组
audio_data = np.array(wavs[2])

# 确保音频数据是一维数组
audio_data = audio_data.flatten()

# 确保音频数据在-1.0到1.0之间
audio_data = np.clip(audio_data, -1.0, 1.0)

# 将音频数据转换为int16类型
audio_data = (audio_data * 32767).astype(np.int16)

# 设置采样率为24,000 Hz
rate = 24000

# 将音频数据保存为WAV文件
output_file = "output_audio.wav"
write(output_file, rate, audio_data)
print(f"Audio file saved as {output_file}")

NameError: name 'wavs' is not defined

### Custom params

In [ ]:
wav = chat.infer('Do you prefer to travel by bike or by bus?', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

In [ ]:
Audio(wav[0], rate=24_000, autoplay=True)

搞一个网上嵌入的pt

In [ ]:
import torch
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}
# 加载保存的嵌入向量
embedding = torch.load('dayoutts.pt')

# 确认嵌入向量的维度是否正确
assert embedding.shape[0] == 768, "Embedding dimension mismatch!"  # 假设你的目标维度是768
params_infer_code={'spk_emb':embedding}
wav = chat.infer('大家好,我是大尤,这是一条用chat T T S合成出来的语音.我只是单纯的想表达一下,[laugh]承峰是真的挺牛的.', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)
Audio(wav[0], rate=24_000, autoplay=True)

INFO:ChatTTS.core:All initialized.
 21%|██        | 432/2048 [00:11<00:42, 38.41it/s]


### fix random speaker

In [ ]:
#采样一个说话者
rand_spk = chat.sample_random_speaker()
# print(rand_spk)  # 打印并保存嵌入向量

# 将嵌入向量保存到文件中
with open('saved_speaker_embedding.txt', 'w') as f:
    f.write(','.join(map(str, rand_spk.tolist())))
print(f"Speaker embedding saved as saved_speaker_embedding.txt")


INFO:ChatTTS.core:All initialized.


Speaker embedding saved as saved_speaker_embedding.txt


  8%|▊         | 169/2048 [00:03<00:44, 42.64it/s]


这个是加载官方的随机向量

In [ ]:
# 从文件中加载嵌入向量
with open('saved_speaker_embedding.txt', 'r') as f:
    saved_spk = list(map(float, f.read().strip().split(',')))
params_infer_code = {'spk_emb' : torch.tensor(saved_spk), }

这个是加载大尤的声音向量

In [ ]:
with open('saved_speaker_embedding_get.txt', 'r') as f:
    saved_spk = list(map(float, f.read().strip().split(',')))
# 将加载的嵌入向量转换为tensor
import torch
fixed_spk = torch.tensor(saved_spk)


import torch.nn as nn

class EmbeddingAdapter(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EmbeddingAdapter, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# 初始化适配器层，将256维嵌入向量转换为768维
adapter = EmbeddingAdapter(input_dim=256, output_dim=768)

# 将适配器层应用到嵌入向量
fixed_spk_adapted = adapter(fixed_spk.unsqueeze(0)).squeeze(0)

params_infer_code = {'spk_emb' : fixed_spk_adapted, }
# # print(rand_spk)
# wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', \
#     params_refine_text=params_refine_text, params_infer_code=params_infer_code)


In [ ]:
wav = chat.infer('大家好,我是大尤,这是一条用chat T T S合成出来的语音.', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)
Audio(wav[0], rate=24_000, autoplay=True)

INFO:ChatTTS.core:All initialized.


 12%|█▏        | 247/2048 [00:05<00:43, 41.55it/s]


In [ ]:
#采样一个说话者
rand_spk = chat.sample_random_speaker()
# print(rand_spk)  # 打印并保存嵌入向量

# 将嵌入向量保存到文件中
with open('saved_speaker_embedding.txt', 'w') as f:
    f.write(','.join(map(str, rand_spk.tolist())))
print(f"Speaker embedding saved as saved_speaker_embedding.txt")
# 从文件中加载嵌入向量
with open('saved_speaker_embedding.txt', 'r') as f:
    saved_spk = list(map(float, f.read().strip().split(',')))
params_infer_code = {'spk_emb' : torch.tensor(saved_spk), }
wav = chat.infer('大家好,我是大尤,这是一条用chat T T S合成出来的语音.', \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)
Audio(wav[0], rate=24_000, autoplay=True)

INFO:ChatTTS.core:All initialized.


Speaker embedding saved as saved_speaker_embedding.txt


 14%|█▍        | 295/2048 [00:07<00:43, 40.29it/s]


In [ ]:
# 保存音频文件
import numpy as np
from scipy.io.wavfile import write

# 假设wavs[0]是包含音频数据的二维数组
audio_data = np.array(wav[0])

# 确保音频数据是一维数组
audio_data = audio_data.flatten()

# 确保音频数据在-1.0到1.0之间
audio_data = np.clip(audio_data, -1.0, 1.0)

# 将音频数据转换为int16类型
audio_data = (audio_data * 32767).astype(np.int16)

# 设置采样率为24,000 Hz
rate = 24000

# 将音频数据保存为WAV文件
output_file = "output_audio.wav"
write(output_file, rate, audio_data)
print(f"Audio file saved as {output_file}")

Audio file saved as output_audio.wav


### Two stage control

In [ ]:
text = "So we found being competitive and collaborative was a huge way of staying motivated towards our goals, so one person to call when you fall off, one person who gets you back on then one person to actually do the activity with."
chat.infer(text, refine_text_only=True)

INFO:ChatTTS.core:All initialized.
 23%|██▎       | 87/384 [00:00<00:01, 150.60it/s]


['so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.']

In [ ]:
text = 'so we found being competitive and collaborative [uv_break] was a huge way of staying [uv_break] motivated towards our goals, [uv_break] so [uv_break] one person to call [uv_break] when you fall off, [uv_break] one person who [uv_break] gets you back [uv_break] on then [uv_break] one person [uv_break] to actually do the activity with.'
wav = chat.infer(text, skip_refine_text=True)

INFO:ChatTTS.core:All initialized.
 49%|████▊     | 995/2048 [00:07<00:07, 141.85it/s]


## LLM Call

In [ ]:
from ChatTTS.experimental.llm import llm_api

API_KEY = ''
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [ ]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了, 有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片、串串香、龙抄手、宜宾燃面、乐山钵钵鸡等, 每样都让人垂涎三尺。


INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片、串串香、龙抄手、宜宾燃面、乐山钵钵鸡等，每样都让人垂涎三尺。


In [ ]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.infer(text, params_infer_code=params_infer_code)

INFO:ChatTTS.core:All initialized.
 32%|███▏      | 647/2048 [00:04<00:09, 140.27it/s]
